### Konstantina Andronikou 

## This notebook is testing the conjuction "and"

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import csv

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
# To remove all warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Someone killed Mary Smith and left.") #In this case ARG1 is the target
output

{'verbs': [{'verb': 'killed',
   'description': '[ARG0: Someone] [V: killed] [ARG1: Mary Smith] and left .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']},
  {'verb': 'left',
   'description': '[ARG0: Someone] killed Mary Smith and [V: left] .',
   'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}],
 'words': ['Someone', 'killed', 'Mary', 'Smith', 'and', 'left', '.']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone killed Mary Smith and left."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'killed',
     'description': '[ARG0: Someone] [V: killed] [ARG1: Mary Smith] and left .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']},
    {'verb': 'left',
     'description': '[ARG0: Someone] killed Mary Smith and [V: left] .',
     'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}],
   'words': ['Someone', 'killed', 'Mary', 'Smith', 'and', 'left', '.']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg1_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name'],meta['last_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [11]:

editor = Editor()

# create examples
t = editor.template('Someone killed {first_name} {last_name} and left.', meta=True, nsamples=1000, remove_duplicates = True)

for k, v in t.items():
    print(k,v)

meta [{'last_name': 'Mitchell', 'first_name': 'Alexandra'}, {'last_name': 'Crawford', 'first_name': 'Rose'}, {'last_name': 'Carter', 'first_name': 'Matt'}, {'last_name': 'Hughes', 'first_name': 'Steven'}, {'last_name': 'Mason', 'first_name': 'Ann'}, {'last_name': 'Russell', 'first_name': 'Richard'}, {'last_name': 'Taylor', 'first_name': 'Johnny'}, {'last_name': 'Bell', 'first_name': 'Sophie'}, {'last_name': 'Carter', 'first_name': 'Brian'}, {'last_name': 'Wood', 'first_name': 'Andrea'}, {'last_name': 'Bailey', 'first_name': 'Suzanne'}, {'last_name': 'Davis', 'first_name': 'Amy'}, {'last_name': 'Hunter', 'first_name': 'Sandra'}, {'last_name': 'Kennedy', 'first_name': 'Steve'}, {'last_name': 'Turner', 'first_name': 'Matt'}, {'last_name': 'James', 'first_name': 'Lauren'}, {'last_name': 'Foster', 'first_name': 'Alfred'}, {'last_name': 'Lewis', 'first_name': 'Scott'}, {'last_name': 'Hamilton', 'first_name': 'Dave'}, {'last_name': 'Hughes', 'first_name': 'Frank'}, {'last_name': 'Ryan', 'firs

In [12]:
with open('Dataset_BiLSTM/With_conjuction.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
test = MFT(**t, name = 'detect_arg1name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/predictions_with_con.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 999 examples
Test cases:      999
Fails (rate):    15 (1.5%)

Example fails:
[ARG0: Someone] [V: killed] [ARG1: Jimmy Moore and left] .
----
[ARG0: Someone] [V: killed] [ARG1: Lawrence Moore and left] .
----
[ARG0: Someone] [V: killed] [ARG1: Howard] [ARGM-TMP: Thompson] and left .
----
['Someone', 'killed', 'Alexandra', 'Mitchell', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Rose', 'Crawford', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Matt', 'Carter', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Steven', 'Hughes', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Ann', 'Mason', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Richard', 'Russell', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Johnny', 'Taylor', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Sophie', 'Bell', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Brian', 'Carter', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Andrea', 'Wood', 'and', 'left', '.'] [ True]
['Som

In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_with_con.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 